In [17]:
print(os.path.join('data_road','training'))
os.path.exists(os.path.join('data_road','training', 'image_2'))

data_road/training


True

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, LSTM, MaxPool2D, Flatten, InputLayer, Reshape, TimeDistributed
from tensorflow.keras.models import Sequential
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data_path = os.path.join('data_road','training')
training_data_path = os.path.join(data_path, "image_2")
label_data_path = os.path.join(data_path, "gt_image_2")


def create_model():
    model = Sequential([
        InputLayer(input_shape=(160,600,5)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (1,5), strides=(1,4), activation='relu'),
        Conv2D(1, (1,4), activation='relu'),
        Flatten(),
        Dense(600, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def crop_image(image):
    """
    Crop center 600x160 of the image and recale pixel values to [0,1]
    """
    height, width, _ = image.shape
    ymin = int(height/2-80)
    ymax = int(height/2+80)
    xmin = int(width/2-300)
    xmax = int(width/2+300)
    cropped = image[ymin:ymax, xmin:xmax]
    cropped = cropped.astype('float32')
    cropped /= 255.0

    return cropped

def resize_image(image):
    """
    Resize image to 600x160 and recale pixel values to [0,1]
    """
    resized = cv2.resize(image, dsize=(600, 160), interpolation=cv2.INTER_CUBIC)
    resized = resized.astype('float32')
    resized /= 255.0
    return resized

In [20]:
names_df = pd.DataFrame(columns=('image_path', 'label_path'))
image_paths = os.listdir(training_data_path)

for image_filename in image_paths:
    x = image_filename.split("_")
    label_path = "_".join([x[0], "road", x[1]])
    dict1 = {'image_path': image_filename, 'label_path': label_path}
    names_df = names_df.append(dict1, ignore_index = True)
    


/var/folders/37/7v3d6_m94rb0x8dq35phqh8m0000gn/T/ipykernel_11652/2788627180.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df = names_df.append(dict1, ignore_index = True)
/var/folders/37/7v3d6_m94rb0x8dq35phqh8m0000gn/T/ipykernel_11652/2788627180.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df = names_df.append(dict1, ignore_index = True)
/var/folders/37/7v3d6_m94rb0x8dq35phqh8m0000gn/T/ipykernel_11652/2788627180.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df = names_df.append(dict1, ignore_index = True)
/var/folders/37/7v3d6_m94rb0x8dq35phqh8m0000gn/T/ipykernel_11652/2788627180.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [23]:
# Prepare training data

# Get all filenames
image_paths = os.listdir(training_data_path)

plot_train_images = False
resized_input_image_array = []
cropped_input_image_array = []

counter = 0
for idx, values in names_df.iterrows():
    image_filename = values.image_path
    counter += 1
    if counter % 20 == 0:
        print("Processed {} of {} images".format(counter, len(names_df.index)))
    image_path = os.path.join(training_data_path, image_filename)

    # Load image and create resized and cropped versions
    image = cv2.imread(image_path)
    resized = resize_image(image)
    cropped = crop_image(image)

    # Prepare the 4th and 5th dimensions which will be horizontal and verticles indices respectively
    # Refer to https://arxiv.org/pdf/1808.04450.pdf section "IV.B.1 Preprocessing"
    rows = np.ndarray([160,600])
    for i in range(160):
        rows[i, :] = i+1  * np.ones([600])
    columns = np.ndarray([160,600])
    for i in range(600):
        columns[:, i] = i+1 * np.ones([160])
    rows = np.expand_dims(rows, 2)
    columns = np.expand_dims(columns, 2)

    # Add row and column dimensions to our images to create 600x160x5 input tensors
    image_tensor_resized = np.concatenate([resized, rows, columns], axis=2)
    image_tensor_cropped = np.concatenate([cropped, rows, columns], axis=2)

    # Plot images if boolean above is set to True
    # Should only do this if processing a couple of images otherwise matplotlip won't be happy with YOU!
    if plot_train_images:
        fig, (ax1, ax2) = plt.subplots(1,2)
        ax1.imshow(image_tensor_resized[:,:,0:3])
        ax1.set_title("resized")
        ax2.imshow(image_tensor_cropped[:,:,0:3])
        ax2.set_title("cropped")
        
    # Add the input tensors to the data arrays
    resized_input_image_array.append(image_tensor_resized)
    cropped_input_image_array.append(image_tensor_cropped)



Processed 20 of 289 images
Processed 40 of 289 images
Processed 60 of 289 images
Processed 80 of 289 images
Processed 100 of 289 images
Processed 120 of 289 images
Processed 140 of 289 images
Processed 160 of 289 images
Processed 180 of 289 images
Processed 200 of 289 images
Processed 220 of 289 images
Processed 240 of 289 images
Processed 260 of 289 images
Processed 280 of 289 images


In [26]:
# Prepare training labels

# Get all filenames
label_paths = os.listdir(label_data_path)

plot_label_images = False
resized_label_array = []
cropped_label_array = []

# Loop through all images in the labeled picture directory
counter = 0
for idx, values in names_df.iterrows():
    image_filename = values.label_path
#     print(label_image)
#     print("road" not in label_image)
#     if "road" not in label_image:
#         print("skipping")
#         continue
    counter += 1
    if counter % 20 == 0:
        print("Processed {} of {} labels".format(counter, len(names_df.index)))
    
    # Load the image and create a resized and cropped version
    image_path = os.path.join(label_data_path, image_filename)
    image = cv2.imread(image_path)
    resized = resize_image(image)
    cropped = crop_image(image)

    # Initialize the 600x1x1 ndarray to be used as the label for this data point
    # The value is the height of the column where the road borders non-road
    resized_label_img = np.ndarray([600])
    cropped_label_img = np.ndarray([600])

    # Loop through each column of the image to see where the road segmentation begins in the label
    for i in range(resized.shape[1]):
        # Check the through the resized image to see where the segmentation begins
        for j in range(resized.shape[0]):
            if np.array_equiv(resized[j,i],[1.,0.,1.]):
                break
        resized_label_img[i] = j+1
        
        # Check through the cropped image to see where the segmentation begins
        for j in range(resized.shape[0]):
            if np.array_equiv(cropped[j,i],[1.,0.,1.]):
                break
        cropped_label_img[i] = j+1

    # Expand dimensions to turn the 600 to a 600x1x1
    resized_label_img = np.expand_dims(resized_label_img, 1)
    resized_label_img = np.expand_dims(resized_label_img, 2)
    cropped_label_img = np.expand_dims(cropped_label_img, 1)
    cropped_label_img = np.expand_dims(cropped_label_img, 2)

    # Add the 600x1x1 label to the label array
    resized_label_array.append(resized_label_img)
    cropped_label_array.append(cropped_label_img)
    
    # Plot images if boolean above is set to True
    # Should only do this if processing a couple of images otherwise matplotlip won't be happy with YOU, yes YOU!!
    if plot_label_images:
        fig, (ax1) = plt.subplots(1,2)
        ax1.imshow(resized)
        ax1.set_title("resized label")
        ax2.imshow(cropped)
        ax2.set_title("cropped label")
    


Processed 20 of 289 labels
Processed 40 of 289 labels
Processed 60 of 289 labels
Processed 80 of 289 labels
Processed 100 of 289 labels
Processed 120 of 289 labels
Processed 140 of 289 labels
Processed 160 of 289 labels
Processed 180 of 289 labels
Processed 200 of 289 labels
Processed 220 of 289 labels
Processed 240 of 289 labels
Processed 260 of 289 labels
Processed 280 of 289 labels


In [32]:
# Convert data and label arrays to nparrays before feeding it to the models
resized_input_image_array_np = np.array(resized_input_image_array)
resized_input_image_array_np = resized_input_image_array_np.astype('float32')
cropped_input_image_array_np = np.array(cropped_input_image_array)
cropped_input_image_array_np = cropped_input_image_array_np.astype('float32')
resized_label_array_np = np.array(resized_label_array)
cropped_label_array_np = np.array(cropped_label_array).reshape((289,600,1,1))


In [28]:
# Create the models
resized_model = create_model()

# Check dem summaries
# resized_model.summary()


2022-04-23 18:40:32.700327: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-23 18:40:32.700726: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [29]:
# Train dem models'
resized_model.fit(resized_input_image_array_np, resized_label_array_np)


2022-04-23 18:41:37.156446: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-23 18:41:38.674248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-23 18:41:39.608845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-23 18:41:41.003337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-23 18:41:44.856006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-23 18:41:45.853352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 30s 2s/step - loss: 18070.9609 - accuracy: 2.1699e-06


In [30]:
# Create the models
cropped_model = create_model()

# Check dem summaries
# cropped_model.summary()


In [33]:
# Train dem models

cropped_model.fit(cropped_input_image_array_np, cropped_label_array_np)


ValueError: Data cardinality is ambiguous:
  x sizes: 289
  y sizes: 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 6